In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, LSTM

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2025-04-26 23:49:40.934938: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745711381.185586      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745711381.257189      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/kaggle/input/house-rent-prediction-dataset/Dataset Glossary.txt
/kaggle/input/house-rent-prediction-dataset/House_Rent_Dataset.csv


In [2]:
house_rent_data = pd.read_csv('/kaggle/input/house-rent-prediction-dataset/House_Rent_Dataset.csv')

In [3]:
house_rent_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4746 entries, 0 to 4745
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Posted On          4746 non-null   object
 1   BHK                4746 non-null   int64 
 2   Rent               4746 non-null   int64 
 3   Size               4746 non-null   int64 
 4   Floor              4746 non-null   object
 5   Area Type          4746 non-null   object
 6   Area Locality      4746 non-null   object
 7   City               4746 non-null   object
 8   Furnishing Status  4746 non-null   object
 9   Tenant Preferred   4746 non-null   object
 10  Bathroom           4746 non-null   int64 
 11  Point of Contact   4746 non-null   object
dtypes: int64(4), object(8)
memory usage: 445.1+ KB


In [4]:
house_rent_data.head()

,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact
0,2022-05-18,2,10000,1100,Ground out of 2,Super Area,Bandel,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner
1,2022-05-13,2,20000,800,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
2,2022-05-16,2,17000,1000,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
3,2022-07-04,2,10000,800,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner
4,2022-05-09,2,7500,850,1 out of 2,Carpet Area,South Dum Dum,Kolkata,Unfurnished,Bachelors,1,Contact Owner


In [5]:
house_rent_data["Furnishing Status"].value_counts()

Furnishing Status
Semi-Furnished    2251
Unfurnished       1815
Furnished          680
Name: count, dtype: int64

In [6]:
#Checking the mean, median, highest and lowest rent
print(f'Highest Rent : {house_rent_data["Rent"].max()}')
print(f'Mean Rent: {house_rent_data["Rent"].mean()}')
print(f'Median Rent : {house_rent_data["Rent"].median()}')
print(f'Lowest Rent : {house_rent_data["Rent"].min()}')

Highest Rent : 3500000
Mean Rent: 34993.45132743363
Median Rent : 16000.0
Lowest Rent : 1200


Now let’s have a look at the rent of the houses in different cities according to the number of bedrooms, halls, and kitchens:

In [7]:
fig = px.bar(house_rent_data, x='City', y='Rent',color='BHK',
             title= 'Rent in different cities based on number of Bedrooms, Halls and Kitchen')
fig.show()

Visualizing rent in different cities accoeding to Area Type

In [8]:
fig = px.bar(house_rent_data, x='City',y='Rent',color='Area Type', 
             title='Rent in different Cities based on Area Type')
fig.show()

In [ ]:
fig = px.bar(house_rent_data, x="City", 
                y = "Rent", 
                color = "Furnishing Status",
            title="Rent in Different Cities based on Furnishing Status")
fig.show()

In [ ]:
fig = px.bar(house_rent_data, x="City", 
                y = "Rent", 
                color = "Size",
            title="Rent in Different Cities based on Apartment Size")
fig.show()

In [ ]:
cities = house_rent_data['City'].value_counts()
label = cities.index
counts = cities.values
colors =['gold', 'lightgreen']

In [ ]:
fig = go.Figure(data = go.Pie(labels= label,values=counts,hole=0.5))
fig.update_layout(title_text = 'Number of Houses Available for Rent')
fig.update_traces(hoverinfo='label+percent',textinfo='value',textfont_size=30,
                 marker= dict(colors=colors, line= dict(color='black', width=3)))

In [ ]:
tenant =house_rent_data['Tenant Preferred'].value_counts()
label= tenant.index
counts = tenant.values
colors = ['gold','lightgreen']

In [ ]:
fig= go.Figure(data = [go.Pie(labels= label, values= counts, hole = 0.5)] )
fig.update_layout(title_text = 'Preference of Tenant in India')
fig.update_traces(hoverinfo= 'label+percent', textinfo= 'value',textfont_size = 30,
                 marker = dict(colors=colors,line=dict(color= 'black',width=3)))

House Rent Prediction Model

In [ ]:
house_rent_data['Area Type'].value_counts()

In [ ]:
house_rent_data['Area Type'] = house_rent_data['Area Type'].map({"Super Area":1,
                                                                "Carpet Area":2,
                                                                "Built Area":3})

In [ ]:
house_rent_data['Area Type'].value_counts()

In [ ]:
house_rent_data['City'].value_counts()

In [ ]:
house_rent_data['City'] = house_rent_data['City'].map({"Mumbai":4000,"Chennai":6000,
                                                      "Bangalore":5600, "Hyderabad":5000,"Delhi":1100,
                                                      "Kolkata":7000})

In [ ]:
house_rent_data['City'].value_counts()

In [ ]:
house_rent_data['Furnishing Status'] = house_rent_data['Furnishing Status'].map({"Unfurnished":0, 
                                                                                 "Semi-Furnished":1,"Furnished":2})

In [ ]:
house_rent_data['Furnishing Status'].value_counts()

In [ ]:
house_rent_data['Tenant Preferred'].value_counts() 

In [ ]:
house_rent_data['Tenant Preferred'] = house_rent_data['Tenant Preferred'].map({"Bachelors":1,"Bachelors/Family":2,"Family":3})

In [ ]:
house_rent_data['Tenant Preferred'].value_counts()

In [ ]:
house_rent_data.head()

In [ ]:
# #Splitting Area
# x= np.array(house_rent_data[['BHK','Size','Area Type','City','Furnishing Status','Tenant Preferred','Bathroom']])
# y= np.array()

In [ ]:
#Drop some columns
house_rent = house_rent_data.drop(columns=["Posted On",'Floor','Area Locality','Point of Contact'])

In [ ]:
x= house_rent.drop(columns = ['Rent'])

In [ ]:
y= house_rent["Rent"]

In [ ]:
X_train,X_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state= 42)

In [33]:
model = Sequential()
model.add(LSTM(128, return_sequences= True,input_shape= (X_train.shape[1],1)))
model.add(LSTM(64,return_sequences = False))
model.add(Dense(25))
model.add(Dense(1))
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                        │ (None, 7, 128)              │          66,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 25)                  │           1,625 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              26 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 117,619 (459.45 KB)

 Trainable params: 117,619 (459.45 KB)

 Non-trainable params: 0 (0.00 B)

In [34]:
model.compile(optimizer = 'adam',loss='mean_squared_error')
model.fit(X_train,y_train, batch_size=1, epochs=21)

Epoch 1/21
3796/3796 ━━━━━━━━━━━━━━━━━━━━ 28s 6ms/step - loss: 9472931840.0000
Epoch 2/21
3796/3796 ━━━━━━━━━━━━━━━━━━━━ 24s 6ms/step - loss: 7762687488.0000
Epoch 3/21
3796/3796 ━━━━━━━━━━━━━━━━━━━━ 24s 6ms/step - loss: 4897024512.0000
Epoch 4/21
3796/3796 ━━━━━━━━━━━━━━━━━━━━ 24s 6ms/step - loss: 5100921344.0000
Epoch 5/21
3796/3796 ━━━━━━━━━━━━━━━━━━━━ 24s 6ms/step - loss: 3971727104.0000
Epoch 6/21
3796/3796 ━━━━━━━━━━━━━━━━━━━━ 25s 6ms/step - loss: 5509445120.0000
Epoch 7/21
3796/3796 ━━━━━━━━━━━━━━━━━━━━ 25s 6ms/step - loss: 6160949248.0000
Epoch 8/21
3796/3796 ━━━━━━━━━━━━━━━━━━━━ 25s 7ms/step - loss: 9238653952.0000
Epoch 9/21
3796/3796 ━━━━━━━━━━━━━━━━━━━━ 25s 7ms/step - loss: 11195661312.0000
Epoch 10/21
3796/3796 ━━━━━━━━━━━━━━━━━━━━ 25s 7ms/step - loss: 3196119552.0000
Epoch 11/21
3796/3796 ━━━━━━━━━━━━━━━━━━━━ 25s 7ms/step - loss: 3880815616.0000
Epoch 12/21
3796/3796 ━━━━━━━━━━━━━━━━━━━━ 25s 7ms/step - loss: 3712219136.0000
Epoch 13/21
3796/3796 ━━━━━━━━━━━━━━━━━━━━ 25s 7

In [35]:
print("Enter House Details to Predict Rent")
a = int(input("Number of Bedroom, House and Kitchen: "))
b = int(input("Size of the House: "))
c = int(input("Area Type (Super Area = 1, Carpet Area = 2, Built Area = 3): "))
d = int(input("Pin Code of the City: "))
e = int(input("Furnishing Status of the House (Unfurnished = 0, Semi-Furnished = 1, Furnished = 2): "))
f = int(input("Tenant Type (Bachelors = 1, Bachelors/Family = 2, Only Family = 3): "))
g = int(input("Number of bathrooms: "))
features = np.array([[a, b, c, d, e, f, g]])
print("Predicted House Price = ", model.predict(features))

Enter House Details to Predict Rent


Number of Bedroom, House and Kitchen:  3
Size of the House:  1100
Area Type (Super Area = 1, Carpet Area = 2, Built Area = 3):  1
Pin Code of the City:  3
Furnishing Status of the House (Unfurnished = 0, Semi-Furnished = 1, Furnished = 2):  2
Tenant Type (Bachelors = 1, Bachelors/Family = 2, Only Family = 3):  3
Number of bathrooms:  2


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 787ms/step
Predicted House Price =  [[26790.156]]
